### Import Required Packages

In [ ]:
import os
import matplotlib.pyplot as plt
import pybedtools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

### Script to generate fragment length distribution plots using kernel density estime for the fraction of reads

In [ ]:
# Path to directory containing the bed files (not downsampled)
input_dir = '/home/jupyter/wb_exp-940-967/cs15_ficoll/cs15_ficoll_bed_files/original_bed_files'

# Initialize empty list to store filenames
file_names = []

# Listing files with progress statements
print("Listing files in directory...")
for filename in os.listdir(directory):
    if filename.endswith('.bed'):
        file_names.append(filename[:-4])  # this remove the .bed extension
print("Files listed.")


# Initialize empty dictionary KDE data for each file
kde_dict = {}

if file_names:
    # Group files based on sample ID (prefix)
    prefix_groups = {}
    for filename in file_names:
        prefix = filename.split('_')[0]  # Extract prefix
        if prefix not in prefix_groups:
            prefix_groups[prefix] = []
        prefix_groups[prefix].append(filename)
    
    # Defining line styles and colors in list and dictionary
    line_styles = ['-', '--', '-.', ':']
    colors = {'CS15': '#619CFF', 'Ficoll': '#F8766D'}
    
    for i, (prefix, filenames) in enumerate(prefix_groups.items()):
        for j, filename in enumerate(filenames):
            # Read in the bed file and convert to a dataframe
            print(f"Processing file {j+1}/{len(filenames)} of group {i+1}/{len(prefix_groups)}: {filename}.bed")
            bed_df = pd.read_csv(os.path.join(directory, filename + '.bed'), sep='\t', header=None)
            
            # Get fragment lengths and total reads
            fragment_lengths = bed_df[2] - bed_df[1]
            total_reads = len(fragment_lengths)
            
            # Assign Donor # to corresponing internal donor ID and assign linestyle
            if prefix == '1982BW':
                donor = 'Donor 1'
                linestyle = line_styles[0]
            elif prefix == '3620BW':
                donor = 'Donor 2'
                linestyle = line_styles[1]
            elif prefix == '2482BW':
                donor = 'Donor 3'
                linestyle = line_styles[2]
            
            # Assign color of linestyle based on condition which is the suffix of filename (CS15 vs Ficoll)
            suffix = filename.split('_')[-1]
            color = colors.get(suffix, 'black')
            
            # kernel density estimate
            kde = gaussian_kde(fragment_lengths)
            x_vals = np.linspace(fragment_lengths.min(), fragment_lengths.max(), 300)
            kde_vals = kde(x_vals)
            
            # normalize the kde values
            kde_vals_normalized = kde_vals / np.sum(kde_vals)
            
            # save the kde values for y-xis
            kde_data[filename] = {'x_vals': x_vals, 'kde_vals_normalized': kde_vals_normalized, 'donor': donor, 'linestyle': linestyle, 'color': color}
            
            print(f"Processed file {j+1}/{len(filenames)} of group {i+1}/{len(prefix_groups)}: {filename}.bed")

# plot
fig, ax = plt.subplots()

for filename, data in kde_data.items():
    ax.plot(data['x_vals'], data['kde_vals_normalized'], label=data['donor'], linestyle=data['linestyle'], color=data['color'])

# Make the legend for linestyle and corresponding donor #
legend_elements = [Line2D([0], [0], linestyle='-', color='black', label='Donor 1'),
                   Line2D([0], [0], linestyle='--', color='black', label='Donor 2'),
                   Line2D([0], [0], linestyle=':', color='black', label='Donor 3')]

# Make the color legend for condition variables
for suffix, color in colors.items():
    legend_elements.append(Line2D([0], [0], linestyle='-', color=color, label=suffix))

ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), title='Legend')

plt.xlabel('Fragment Length')
plt.ylabel('Fraction of Reads')
plt.title('Fragment Length Distribution')
plt.savefig('fragment_length_distribution.svg', bbox_inches='tight')  # Save as SVG with tight bounding box
plt.show()